In [5]:
import gurobipy as gp
from gurobipy import GRB
model = gp.Model("MIP-moo-LAMOSCAD")
# ---------------------------------------- Decision variable -------------------------------------------------------
Z = [1,2,3,4]

deploy = model.addVars(Z, vtype=GRB.CONTINUOUS, lb=0)
    # C10: resource capacity constaint & deploy only when facility is open
C_resource_capacity = model.addConstrs((deploy[z] <= 100 for z in Z), name='C_open_facility')
# C16: deploy less than demand
C_deploy_demand = model.addConstrs((deploy[z] >= 10
                                    for z in Z), name='C_deploy_demand')

model.ModelSense = GRB.MINIMIZE


model.setObjective(gp.quicksum((deploy[z] for z in Z)))
# Write the model
#model.write(f'Outputs/model_moo0012.lp')


In [7]:
model.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 8 rows, 4 columns and 8 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+02]

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  0.000000000e+00


In [8]:

import pandas as pd
# %% Query number of multiple objectives, and number of solutions
x = model.getVars()
deploy_series = pd.Series(model.getAttr('X', deploy))
deploy_series[deploy_series > 0.5]
nSolutions = model.SolCount
nObjectives = model.NumObj
print('Problem has', nObjectives, 'objectives')
print('Gurobi found', nSolutions, 'solutions')
solutions = []
for s in range(nSolutions):
    # Set which solution we will query from now on
    model.params.SolutionNumber = s

    # Print objective value of this solution in each objective
    print('Solution', s, ':', end='')
    for o in range(nObjectives):
        # Set which objective we will query
        model.params.ObjNumber = o
        # Query the o-th objective value
        print(' ', round(model.ObjVal, 2), end=' ')
        # query the full vector of the o-th solution
        solutions.append(model.getAttr('X', x))
    print()

for j in range(len(x)):
    if x[j].X > 0:
        print(x[j].VarName, x[j].X, end=' ')
        print(' ')

Problem has 1 objectives
Gurobi found 1 solutions
Solution 0 :  0.0 
